# Data Engineering Assignment for Rahul Rajput

## Loading Files

In [35]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

1. Load the Analyst Alerts File

2. Load remaining files. Store the headers of each file in a separate list as well, used to compare if all files have the same headers.

3. For each file check if first column has any duplicates or NA

4. If not, make first column index and remove its name 

5. Convert event_date column to datetime

6. Check for duplicates and NAs in each of the files

In [36]:
Analyst_Alerts = pd.read_csv("Jack Gaynor - Energy Fleet Analyst Data Test/Tesla_SEIA_analyst_alerts_data.csv")

In [37]:
print(Analyst_Alerts.iloc[:,0].isna().any())
print(Analyst_Alerts.iloc[:,0].duplicated().any())
Analyst_Alerts = Analyst_Alerts.set_index(Analyst_Alerts.columns[0], inplace=False)
Analyst_Alerts.index.name = None
Analyst_Alerts['event_date'] = pd.to_datetime(Analyst_Alerts['event_date'])

False
False


In [38]:
Analyst_Alerts.head()

,timestamp,SITE,ASSET,COMPONENT,alert_name,event_date
0,1647619928967,-1844894889,1477164192,-1702932383,LM_a141_cableTempImbalance,2022-03-18
1,1647620105662,-1844894889,1477164192,-1702932383,LM_a141_cableTempImbalance,2022-03-18
2,1647259045505,-977145491,-2098644635,-226928606,LM_a141_cableTempImbalance,2022-03-14
3,1647193059705,-207607826,-1470107556,-837209140,LM_a141_cableTempImbalance,2022-03-13
4,1647193502594,-207607826,-1470107556,-837209140,LM_a141_cableTempImbalance,2022-03-13


In [39]:
Analyst_Alerts.dtypes

timestamp              int64
SITE                   int64
ASSET                  int64
COMPONENT              int64
alert_name            object
event_date    datetime64[ns]
dtype: object

In [40]:
%%time
folder_path = "Jack Gaynor - Energy Fleet Analyst Data Test/"
files = [file for file in os.listdir(folder_path) if file not in ['.DS_Store', 'Tesla_SEIA_analyst_alerts_data.csv']]

dataframes = {}
headers = []

for file in files:
    file_path = os.path.join(folder_path, file)
    
    # Storing headers of each file.
    header = pd.read_csv(file_path, nrows=0)
    headers.append(set(header.columns))
    
    # Loading each file.
    df = pd.read_csv(file_path)
    
    # Checking if first column has any duplicates or NAs, if not then make it index.
    first_col_has_duplicates = df.iloc[:, 0].duplicated().any()
    first_col_has_na = df.iloc[:, 0].isna().any()
    
    if not (first_col_has_duplicates or first_col_has_na):
        df = df.set_index(df.columns[0], inplace=False)
        df.index.name = None
        
        # Converting 'event_date' column to datetime format
        df['event_date'] = pd.to_datetime(df['event_date'])
        
        # Checking for duplicates and NAs in each of the files
        duplicates = df.duplicated().any()
        NAs = df.isna().any().any()
        
        if duplicates or NAs:
            if duplicates:
                print(f"There are duplicates in {file}")
                
            if NAs:
                print(f"There are NAs in {file}")
        else:
            # Adding DataFrame to dictionary 'dataframes'
            dataframes[file] = df
    else:
        print(f"First column of {file} contains duplicates or NAs")

# dataframes dictionary contains 32 files

CPU times: user 13.5 s, sys: 1.12 s, total: 14.6 s
Wall time: 16.2 s


In [41]:
len(dataframes)

32

In [42]:
dataframes["Tesla_SEIA_analyst_signals_data_'2022-03-12'_'LM_pcbaTempDegC'.csv"].dtypes

timestamp               int64
SITE                    int64
ASSET                   int64
COMPONENT               int64
signal_name            object
VALUE                 float64
event_date     datetime64[ns]
dtype: object

In [43]:
if all(set(header) == set(headers[0]) for header in headers):
    print("All headers are the same.")
else:
    print("Headers are not the same.")

All headers are the same.


In [44]:
%%time
# Checking for duplicate records across the 32 files.
# Concatenating all DataFrames into a single DataFrame.
combined_df = pd.concat(dataframes.values(), ignore_index=True)
duplicate_records = combined_df[combined_df.duplicated(keep=False)]

if not duplicate_records.empty:
    print("There are duplicate records across different files:")
    print(duplicate_records)
else:
    print("No duplicate records found across different files.")

No duplicate records found across different files.
CPU times: user 6.38 s, sys: 2.05 s, total: 8.43 s
Wall time: 9.61 s


In [32]:
# temp_df = pd.DataFrame()
# non_temp_df = pd.DataFrame()

# for key, df in dataframes.items():
#     if 'Temp' in key:
#         temp_df = pd.concat([temp_df, df])
#     else:
#         non_temp_df = pd.concat([non_temp_df, df])

In [33]:
# temp_df.to_csv('Temperatures.csv')
# non_temp_df.to_csv('Powers.csv')

In [34]:
fig = px.box(temp_df, y="VALUE", title="Boxplot of Temperatures", color_discrete_sequence=["blue"])
fig.show()

NameError: name 'temp_df' is not defined

In [ ]:
fig = px.box(non_temp_df, y="VALUE", title="Boxplot of Power", color_discrete_sequence=["blue"])
fig.show()

#### Analyst Alerts and the remaining 32 Temperature and Power files have been checked and loaded

--------------------------------------------------------------------------------------------------------

## Question 1 and 2 - Preparation

####  From the dataframes dictionary created at the start, we create 8 new dictionaries - one for each date of records present. Each date has 4 files: 3 related to Temperature logs and 1 related to PowerLimit logs. The new dictionaries also create shortened names for the files.

In [45]:
%%time
import re
from collections import defaultdict

# Extracting date from the filename
def extract_date(file_name):
    return re.search(r"'(\d{4}-\d{2}-\d{2})'", file_name).group(1)

# Shortening Keys
def shorten_key(key):
    return key.split("_")[-1].split(".")[0].rstrip("'")

date_grouped_dataframes = defaultdict(dict)

for file_name, table in dataframes.items():
    date = extract_date(file_name)
    shortened_name = shorten_key(file_name)
    date_grouped_dataframes[date][shortened_name] = table

# Creating dictionaries for each date
dataframes_12 = date_grouped_dataframes['2022-03-12']
dataframes_13 = date_grouped_dataframes['2022-03-13']
dataframes_14 = date_grouped_dataframes['2022-03-14']
dataframes_15 = date_grouped_dataframes['2022-03-15']
dataframes_16 = date_grouped_dataframes['2022-03-16']
dataframes_17 = date_grouped_dataframes['2022-03-17']
dataframes_18 = date_grouped_dataframes['2022-03-18']
dataframes_19 = date_grouped_dataframes['2022-03-19']

CPU times: user 72.6 ms, sys: 256 ms, total: 329 ms
Wall time: 814 ms


#### Data will be prepared in this part to answer Questions 1 and 2. The following steps were taken:

For Question 1: list of components that have an average temperature above 90 degrees during any 30 second period.

Each dataset will undergo a defined procedure:

1. The dataset is sorted by component and timestamp.


The following fields are then calculated by first splitting the dataframe by 'COMPONENT'. The edge cases mentioned are applicable to each COMPONENT group.


2. A new field - Time Interval is created. It is calulated between the current record and the next by taking the difference between the current and preceeding timestamp value. The difference is then divided by 1000 to convert figure into seconds.
(Check code - may need to change a little bit. The averages for the normal values should be calculated by taking the prev value and n-1 future values.)


3. A new field - Window Size is created. It is calculated by summing the current time_interval value and proceeding time interval values till the sum reaches 30 or more. The window size variable allows us to calculate rolling averages of the 'VALUE' field for events which span atleast 30 seconds.


4. A new field - Rolling Average is created. It is calculated by summing the 'VALUE' field over the next n records, where n is the window size, and dividing by n.


5. We still have to deal with the 3 edge cases: The first row, second last row and last row. For the first row there is no time interval value and the rolling average variable is set as the 'VALUE' variable. For the second last row we calculate the rolling average over the 4th last, 3rd last, and 2nd last rows. For the last row the rolling average is calculated over the the last 3 records.


6. We then store each record whose 'VALUE' falls within a 30 second interval where the average temperature exceeds 90.


7. This procedure is applied to all Temperature dataframes to answer Question 1.


This procedure is optimised through parallel processing via the Pathos library.

In [46]:
%%time
# PROCEDURE - calculating the 'Window_Size' and 'Rolling_Avg' fields.

# 'group' represents the dataset filtered by each components. This function will be applied to each of these 
# filtered datasets seprately.

def FindAbove90(group): 
    group = group.reset_index(drop=True)
    
    # Initialising lists to store and append the 3 new calculated fields for each component's records.
    window_sizes = []
    rolling_avgs = []
    records_above_90 = [] # DataFrame storing all relevant records
    timestamp_intervals = []
    
    # Looping over each row within the filtered dataset.
    for i, row in group.iterrows():
        # Calculating Window Sizes
        timesum = row['time_interval'] #variable tracking the sum of time intervals
        cnt = 0
        j = i + 1

        while timesum < 30 and j < len(group): 
            timesum += group.loc[j, 'time_interval']
            cnt += 1
            j += 1
        
        # Edge Case - First Row
        if i == 0:  
            window_sizes.append(0)
            rolling_avg = row['VALUE']
        
        else:
            # Edge Case - Second-last Row
            if i == len(group) - 2:  
                window_sizes.append(4)

            # Edge Case - Last Row
            elif i == len(group) - 1:  
                window_sizes.append(3)

            # All other cases
            else:
                window_sizes.append(cnt + 1)

            # Calculating the Rolling Averages of the 'VALUE' field for each row.
            
            # Edge Case - Last Row
            if i == len(group) - 1:
                rolling_avg = group['VALUE'].iloc[i - 2:i + 1].mean()
                if rolling_avg > 90:
                    records_above_90.extend(group.iloc[i - 2:i + 1].to_dict('records'))

            # Edge Case - Second-last Row
            elif i == len(group) - 2:
                rolling_avg = group['VALUE'].iloc[i - 2:i + 2].mean()
                if rolling_avg > 90:
                    records_above_90.extend(group.iloc[i - 2:i + 2].to_dict('records'))

            # All other cases
            elif i + cnt + 1 <= len(group):
                rolling_avg = group['VALUE'].iloc[i-1:i + (cnt + 1) - 1].mean()
                if rolling_avg > 90:
                    records_above_90.extend(group.iloc[i-1:i + (cnt + 1) - 1].to_dict('records'))
            else:
                rolling_avg = np.nan

        rolling_avgs.append(rolling_avg)

    group['Window_Size'] = window_sizes
    group['Rolling_Avg'] = rolling_avgs
    return group, records_above_90

from pathos.multiprocessing import ProcessingPool as Pool

def process_group(group):
    return FindAbove90(group)

# Step 1: Get and store the "records_above_90_df" for each of the files related to temperature
def get_records_above_90(df):
    with Pool() as p:
        results = p.map(process_group, [group for _, group in df.groupby('COMPONENT')])

    _, records_above_90 = zip(*results)

    # Flatten records_above_90 list
    records_above_90 = [record for sublist in records_above_90 for record in sublist]

    # Create a DataFrame from the records_above_90 list
    records_above_90_df = pd.DataFrame(records_above_90)
    
    return records_above_90_df

CPU times: user 26.1 ms, sys: 57.2 ms, total: 83.4 ms
Wall time: 614 ms


## Question 1 and 2 - Answers

#### For Question 1:

1. For each date, we identify the files corersponding to temperature logs and identify the records where the average temperature for a component was above 90 degrees and store them in a global dataframe 'Above90_Temp'. (global implying across all dates.)


2. We then generate a list of unique components present in the above dataframe to identify all components that had an average temperature above 90 degrees during any 30 second period.

In [47]:
%%time

## QUESTION 1

def timeinterval(row):
    row['time_interval'] = row['timestamp'].diff()/1000
    return row

concatenated_Above90 = pd.DataFrame()

cnt = 12
for date_dict in [dataframes_12, dataframes_13, dataframes_14, dataframes_15, dataframes_16, dataframes_17, dataframes_18, dataframes_19]:
    updated_dict = {}
    for key, df in date_dict.copy().items():
        if "Temp" in key:
            print(cnt)
            print(key)

            updated_dict[key] = df.groupby('COMPONENT').apply(timeinterval).reset_index(drop=True)
            concatenated_Above90 = pd.concat([concatenated_Above90, get_records_above_90(updated_dict[key])])
    
    cnt += 1
    
TimeStamps_df = pd.DataFrame(concatenated_TimeStamps, columns=['start_timestamp', 'end_timestamp'])

12
pcbaTempDegC


<timed exec>:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


12
handleNegCoreTempDegC


<timed exec>:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


12
handlePosCoreTempDegC


...

19
handleNegCoreTempDegC


<timed exec>:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


19
handlePosCoreTempDegC


<timed exec>:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


NameError: name 'concatenated_TimeStamps' is not defined

In [48]:
concatenated_Above90

,timestamp,SITE,ASSET,COMPONENT,signal_name,VALUE,event_date,time_interval
0,1647125933652,-1492083459,1996369523,-1139065776,LM_handleNegCoreTempDegC,89.253668,2022-03-12,10.009
1,1647125943652,-1492083459,1996369523,-1139065776,LM_handleNegCoreTempDegC,90.556335,2022-03-12,10.000
2,1647125954655,-1492083459,1996369523,-1139065776,LM_handleNegCoreTempDegC,91.434213,2022-03-12,11.003
3,1647125943652,-1492083459,1996369523,-1139065776,LM_handleNegCoreTempDegC,90.556335,2022-03-12,10.000
4,1647125954655,-1492083459,1996369523,-1139065776,LM_handleNegCoreTempDegC,91.434213,2022-03-12,11.003
...,...,...,...,...,...,...,...,...
799,1647715486304,-977145491,-2098644635,2043382078,LM_handlePosCoreTempDegC,89.829678,2022-03-19,9.999
800,1647715496312,-977145491,-2098644635,2043382078,LM_handlePosCoreTempDegC,90.059046,2022-03-19,10.008
801,1647715486304,-977145491,-2098644635,2043382078,LM_handlePosCoreTempDegC,89.829678,2022-03-19,9.999
802,1647715496312,-977145491,-2098644635,2043382078,LM_handlePosCoreTempDegC,90.059046,2022-03-19,10.008


In [49]:
#concatenated_Above90.to_csv('TempAbove90.csv')

In [50]:
component_counts = concatenated_Above90['COMPONENT'].value_counts()
component_freq_table = pd.DataFrame(component_counts).reset_index()
component_freq_table.columns = ['Component', 'Frequency']
output = component_freq_table.sort_values(by='Frequency', ascending=False)
print(output)

     Component  Frequency
0    399812473        601
1   1348730178        567
2  -1473041981        546
3  -1996083301        530
4    851777939        480
5   1036677714        463
6    420204713        317
7  -1139065776        291
8   1212935405        234
9  -1107987050        216
10 -2048121660        189
11   369714770        182
12 -1644468160        178
13  -784889276        159
14 -1745692884        148
15  1395314372        122
16  1220791812        115
17  1130578083        109
18  1060928447        103
19   672028928         91
20  1261478727         90
21 -1822444058         74
22 -1747047773         66
23 -1345639539         60
24 -1112487124         38
25 -1702932383         28
26  -824155902         23
27  1411109599         21
28 -2126979618         18
29  2043382078         18
30  1449025406         13
31 -1489919606          6


#### For Question 2:

We use the global dataframe 'Above90' generated in Question 1, with all records where the average temperature of a component exceeded 90 degrees during any 30 second period, to identify corresponding Power Limit Records.


1. For each date we load the Power Limit dataframe and filter the 'Above90_Temp' dataframe by 'event_date' and then match records from both dataframes.


2. The matching is done on 'timestamp', 'SITE', 'ASSET', and 'COMPONENT' fields. Matching records from each Power Limit file are stored in a global dataframe 'Above90_Power'. The number of records for Temperature files and Power file are different, therefore to allow for the Power records being logged at slightly different timestamps, the matching for timestamp would be done with a margin of 2 seconds from the record where the avg temp was > 90.


3. We then find the Maximum value of 'VALUE' field for each component in the 'Aboive90_Power' dataframe 

In [51]:
## QUESTION 2

dates = ['2022-03-12', '2022-03-13', '2022-03-14', '2022-03-15', '2022-03-16', '2022-03-17', '2022-03-18', '2022-03-19']
powerlimit_above90 = []
cnt = 12
for date_dict, date in zip([dataframes_12, dataframes_13, dataframes_14, dataframes_15, dataframes_16, dataframes_17, dataframes_18, dataframes_19],dates):
    print(cnt)
    # Creating filtered dataframes
    power_limit_df = date_dict['powerLimit']
    unique_components = concatenated_Above90['COMPONENT'].unique()
    power_limit_df = power_limit_df[power_limit_df['COMPONENT'].isin(unique_components)]
    filtered_concatenated_Above90 = concatenated_Above90[concatenated_Above90['event_date']==date]
    #print(filtered_concatenated_Above90.head(5))
    
    # Identifying records in the Power Limit files corresponding to records where the Average temp > 90 
    # for atleast 30 seconds
    for _, record in filtered_concatenated_Above90.iterrows():
        matching_power_limit_record = power_limit_df.loc[(power_limit_df['timestamp'].between(record['timestamp']-2000, record['timestamp']+2000)) & 
                                                          (power_limit_df['SITE'] == record['SITE']) &
                                                          (power_limit_df['ASSET'] == record['ASSET']) &
                                                          (power_limit_df['COMPONENT'] == record['COMPONENT'])]
        if not matching_power_limit_record.empty:
            powerlimit_above90.append(matching_power_limit_record.iloc[0])
    cnt = cnt + 1
powerlimit_above90_df = pd.DataFrame(powerlimit_above90)

12
13
14
15
16
17
18
19


In [52]:
#powerlimit_above90_df.to_csv('PowerAbove90.csv')

In [53]:
powerlimit_above90_df

,timestamp,SITE,ASSET,COMPONENT,signal_name,VALUE,event_date
247419,1647125933642,-1492083459,1996369523,-1139065776,LM_powerLimit,290.118909,2022-03-12
247427,1647125944649,-1492083459,1996369523,-1139065776,LM_powerLimit,286.958752,2022-03-12
247432,1647125955652,-1492083459,1996369523,-1139065776,LM_powerLimit,281.348288,2022-03-12
247427,1647125944649,-1492083459,1996369523,-1139065776,LM_powerLimit,286.958752,2022-03-12
247432,1647125955652,-1492083459,1996369523,-1139065776,LM_powerLimit,281.348288,2022-03-12
...,...,...,...,...,...,...,...
299183,1647715486295,-977145491,-2098644635,2043382078,LM_powerLimit,229.033051,2022-03-19
299187,1647715496312,-977145491,-2098644635,2043382078,LM_powerLimit,226.068743,2022-03-19
299183,1647715486295,-977145491,-2098644635,2043382078,LM_powerLimit,229.033051,2022-03-19
299187,1647715496312,-977145491,-2098644635,2043382078,LM_powerLimit,226.068743,2022-03-19


In [54]:
max_values_component = powerlimit_above90_df.groupby('COMPONENT')['VALUE'].max()
max_values_component_df = pd.DataFrame(max_values_component.reset_index())
print(max_values_component_df)

     COMPONENT       VALUE
0  -2126979618  229.461281
1  -2048121660  274.345267
2  -1996083301  359.295607
3  -1822444058  244.193624
4  -1747047773  267.027372
5  -1745692884  304.524100
6  -1702932383  236.621925
7  -1644468160  278.740096
8  -1489919606  268.480017
9  -1473041981  277.391832
10 -1345639539  230.896838
11 -1139065776  290.118909
12 -1112487124  238.495042
13 -1107987050  308.892027
14  -824155902  204.718435
15  -784889276  271.398547
16   369714770  270.149446
17   399812473  303.394367
18   420204713  297.412315
19   672028928  286.584675
20   851777939  297.148150
21  1036677714  294.268552
22  1060928447  208.680031
23  1130578083  297.288226
24  1212935405  295.949308
25  1220791812  276.828181
26  1261478727  260.273086
27  1348730178  251.414973
28  1395314372  244.694867
29  1411109599  261.185191
30  1449025406  211.765505
31  2043382078  242.334900


--------------------------------------------------------------------------------------------------------

# Question 3

In [55]:
Analyst_Alerts.head()

,timestamp,SITE,ASSET,COMPONENT,alert_name,event_date
0,1647619928967,-1844894889,1477164192,-1702932383,LM_a141_cableTempImbalance,2022-03-18
1,1647620105662,-1844894889,1477164192,-1702932383,LM_a141_cableTempImbalance,2022-03-18
2,1647259045505,-977145491,-2098644635,-226928606,LM_a141_cableTempImbalance,2022-03-14
3,1647193059705,-207607826,-1470107556,-837209140,LM_a141_cableTempImbalance,2022-03-13
4,1647193502594,-207607826,-1470107556,-837209140,LM_a141_cableTempImbalance,2022-03-13


In [56]:
Analyst_Alerts['alert_name'].unique()

array(['LM_a141_cableTempImbalance', 'LM_a247_ambientTempStale',
       'LM_a191_negTempRationality', 'LM_a188_negTempOT',
       'LM_a243_negCoreTempHiFoldBk'], dtype=object)

All alerts are temperature related

In [57]:
Analyst_Alerts = Analyst_Alerts.sort_values(by='event_date')

In [58]:
Analyst_Alerts['event_date'].value_counts()

2022-03-14    660
2022-03-16     51
2022-03-17     42
2022-03-13     30
2022-03-15     30
2022-03-19     13
2022-03-12      4
2022-03-18      2
Name: event_date, dtype: int64

#### For Question 3:

All Alerts in the Analyst Alerts dataframe correspond to Temperature related incidents. Logic for Identifying Temperature Records for each Alert raised:

1. For each date, we start by loading all 3 temperature files and concatenating them into one dataframe. 


2. Filter the Analyst Alerts dateset by date. 


3. We assume that there would be a delay between the Alert firing and the actual Log being generated. Therefore we start by identifying records from the Temperature dataframe which occurred between a range of Alert timestamp - 10 seconds to the Alert timestamp. For cases where there was a match in the logs a definitive spike was observed within 10 seconds.


4. Then for each date we filter the records identified fron the Temperature dataframe by components which are present in the Analyst Alerts dataset for that particular date. 


5. These filtered records are then stored in a global dataframe 'Alert_Matches_Temp' along with the index of the record from the Alert_Analyst for which the records were identified as potential matches.


6. This same process is repeated to identify the records from Power Limit files correponding to Alerts raised. These records are then stored in a global dataframe 'Alert_Matches_Power'


7. To identify the temperature/power log corresponding to the Alert we create a function which takes as input the index of the particular Alert from Analyst Alert file and the dataframe of Matches (Alert_Matches_Temp OR Alert_Matches_Power dataframe).


8. The Matches dataframe is filtered by the index of the Alert. If there are not Matches to the particular Alert index, the function returns None.


9. For the matched records we get for the particular alert, we identify the record with the highest 'VALUE' entry.


10. We also define a threshold value to avoid confusion if there are multiple records whose 'VALUE' entry is similar to the record with the highest 'VALUE' entry. 

    
    In the case where the difference between the Maximum 'VALUE' and the second highest 'VALUE' is above the threshold, the record with the Max 'VALUE' is identified as the record correspondong to the alert. In the case where there is 1 or more records for which the difference between their 'VALUE' entry and the max 'VALUE' entry is under a particular threshold, we still opt to identify the record with the Max 'VALUE'. We cannot assume that the record closest in timestamp would be the one correly corresponding to the Alert either.


In [59]:
%%time
dates = ['2022-03-12', '2022-03-13', '2022-03-14', '2022-03-15', '2022-03-16', '2022-03-17', '2022-03-18', '2022-03-19']
date_dicts = [dataframes_12, dataframes_13, dataframes_14, dataframes_15, dataframes_16, dataframes_17, dataframes_18, dataframes_19]

Alert_Matches_Temp = pd.DataFrame()
Alert_Matches_Power = pd.DataFrame()

for date, date_dict in zip(dates, date_dicts):
    filtered_alerts = Analyst_Alerts[Analyst_Alerts['event_date'] == date]
    concatenated_temp = pd.DataFrame()
    concatenated_power = pd.DataFrame()

    for key in date_dict.keys():
        if "Temp" in key:
            concatenated_temp = pd.concat([concatenated_temp, date_dict[key]])
        elif "Power" in key or "power" in key:
            concatenated_power = pd.concat([concatenated_power, date_dict[key]])
    
    for i, alert_record in filtered_alerts.iterrows():
        matching_temp_record = concatenated_temp.loc[concatenated_temp['timestamp'].between(alert_record['timestamp'] - 10000, alert_record['timestamp'])]
        matching_power_record = concatenated_power.loc[concatenated_power['timestamp'].between(alert_record['timestamp'] - 10000, alert_record['timestamp'])]

        if not matching_temp_record.empty:
            matching_temp_record = matching_temp_record[matching_temp_record['COMPONENT'].isin(filtered_alerts['COMPONENT'])]
            matching_temp_record['alert_index'] = i
            Alert_Matches_Temp = pd.concat([Alert_Matches_Temp, matching_temp_record])

        if not matching_power_record.empty:
            matching_power_record = matching_power_record[matching_power_record['COMPONENT'].isin(filtered_alerts['COMPONENT'])]
            matching_power_record['alert_index'] = i
            Alert_Matches_Power = pd.concat([Alert_Matches_Power, matching_power_record])

Alert_Matches_Temp = Alert_Matches_Temp.reset_index(drop=True)
Alert_Matches_Power = Alert_Matches_Power.reset_index(drop=True)


CPU times: user 3.96 s, sys: 784 ms, total: 4.75 s
Wall time: 6.5 s


In [60]:
print(Alert_Matches_Temp.shape)
print(Alert_Matches_Power.shape)

(2530, 8)
(1636, 8)


In [61]:
Alert_Matches_Temp

,timestamp,SITE,ASSET,COMPONENT,signal_name,VALUE,event_date,alert_index
0,1647112002197,-2073653851,-2126066089,-46972795,LM_handleNegCoreTempDegC,34.799605,2022-03-12,6
1,1647112002197,-2073653851,-2126066089,-46972795,LM_handlePosCoreTempDegC,52.550562,2022-03-12,6
2,1647112093224,-2073653851,-2126066089,-46972795,LM_pcbaTempDegC,31.318638,2022-03-12,7
3,1647112095207,-2073653851,-2126066089,-46972795,LM_handleNegCoreTempDegC,40.097450,2022-03-12,7
4,1647112095207,-2073653851,-2126066089,-46972795,LM_handlePosCoreTempDegC,59.486019,2022-03-12,7
...,...,...,...,...,...,...,...,...
2525,1647724516811,-152918872,365951380,420204713,LM_pcbaTempDegC,55.512052,2022-03-19,107
2526,1647724518488,-1844894889,1477164192,-1702932383,LM_handleNegCoreTempDegC,56.202208,2022-03-19,107
2527,1647724515820,-152918872,365951380,420204713,LM_handleNegCoreTempDegC,52.496182,2022-03-19,107
2528,1647724518488,-1844894889,1477164192,-1702932383,LM_handlePosCoreTempDegC,74.941666,2022-03-19,107


#### Identifying which Alert records were not matched in the Temperature logs

In [62]:
all_missing_indexes_power = []

# Iterate over the unique dates in Analyst_Alerts
for date in Analyst_Alerts['event_date'].unique():
    # Get the indexes of Analyst_Alerts records for the current date
    analyst_alerts_indexes = set(Analyst_Alerts[Analyst_Alerts['event_date'] == date].index)

    # Get the alert_index values from Alert_Matches_Temp for the current date
    alert_matches_temp_indexes = set(Alert_Matches_Power[Alert_Matches_Power['event_date'] == date]['alert_index'])
    missing_indexes = analyst_alerts_indexes - alert_matches_temp_indexes 
    all_missing_indexes_power.extend(missing_indexes)


In [63]:
len(all_missing_indexes_power)

205

In [64]:
all_missing_indexes = []

# Iterate over the unique dates in Analyst_Alerts
for date in Analyst_Alerts['event_date'].unique():
    # Get the indexes of Analyst_Alerts records for the current date
    analyst_alerts_indexes = set(Analyst_Alerts[Analyst_Alerts['event_date'] == date].index)

    # Get the alert_index values from Alert_Matches_Temp for the current date
    alert_matches_temp_indexes = set(Alert_Matches_Temp[Alert_Matches_Temp['event_date'] == date]['alert_index'])
    missing_indexes = analyst_alerts_indexes - alert_matches_temp_indexes
    all_missing_indexes.extend(missing_indexes)

In [65]:
len(all_missing_indexes)

212

In [66]:
# Get the unique dates and corresponding unique COMPONENT values from the missing records
unique_dates_components = Analyst_Alerts.iloc[all_missing_indexes,:].drop_duplicates()

# Iterate through the unique dates and COMPONENT values
for _, row in unique_dates_components.iterrows():
    event_date = row['event_date'].strftime('%Y-%m-%d')
    #print(event_date)
    component = row['COMPONENT']
    
    # Create a combined DataFrame for the temperature dataframes for the current date
    concat_test = pd.DataFrame()
    
    for key in date_grouped_dataframes[event_date]:
        if "Temp" in key:
            concat_test = pd.concat([concat_test, date_grouped_dataframes[event_date][key]], ignore_index=True)
    
    # If current unique COMPONENT value is in the combined DataFrame
    component_exists = concat_test['COMPONENT'].isin([component]).any()
    
    if not component_exists:
        print(f"Date: {event_date}, COMPONENT: {component}")

Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: 1105510009
Date: 2022-03-16, COMPONENT: 1105510009
Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: 1105510009
Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: 1105510009
Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: 1105510009
Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -1843899103
Date: 2022-03-16, COMPONENT: -404457910
Date: 2022-03-16, COMPONENT: 1105510009
Date: 2022-03-17, COMPONENT: -404457910
Date: 2022-03-19, COMPONENT: -1843899103
Date: 2022-03-17, COMPONENT: -358949053


We observe that out of the 832 Alerts, we were not able to identify temperature logs for 212 Alerts. This is in part due to certain components present in Alerts dataframe not being present in the temperature files.

In [67]:
def HighestValue(analyst_alert_index, alert_matches_df, value_tolerance=4):
    matching_records = alert_matches_df[alert_matches_df['alert_index'] == analyst_alert_index]

    if matching_records is None:
        return None

    highest_value = matching_records['VALUE'].max()
    close_values = matching_records[abs(matching_records['VALUE'] - highest_value) <= value_tolerance]

    if len(close_values) == 1:
        return close_values.iloc[0]
    else:
        return close_values.iloc[0]
#         analyst_alert_timestamp = Analyst_Alerts.loc[analyst_alert_index, 'timestamp']
#         close_values['time_diff'] = abs(close_values['timestamp'] - analyst_alert_timestamp)
#         result = close_values.loc[close_values['time_diff'].idxmin()]
#         return result.drop('time_diff')

In [68]:
analyst_alert_index = 1  # Change this to the index of the Analyst Alert record you're interested in
highest_value_temp_record = HighestValue(analyst_alert_index, Alert_Matches_Temp)
highest_value_power_record = HighestValue(analyst_alert_index, Alert_Matches_Power)

print("Highest value Temp record:")
print(highest_value_temp_record)
print()
print("Highest value Power record:")
print(highest_value_power_record)

Highest value Temp record:
timestamp                 1647620101684
SITE                        -1844894889
ASSET                        1477164192
COMPONENT                   -1702932383
signal_name    LM_handleNegCoreTempDegC
VALUE                         74.831863
event_date          2022-03-18 00:00:00
alert_index                           1
Name: 2477, dtype: object

Highest value Power record:
timestamp            1647620097571
SITE                   -1844894889
ASSET                   1477164192
COMPONENT              -1702932383
signal_name          LM_powerLimit
VALUE                   130.468721
event_date     2022-03-18 00:00:00
alert_index                      1
Name: 1600, dtype: object


#### Identifying the matched records for each Alert index in Alert_Matches_Temp and Alert_Matches_Power. There will be 627 records for Temperature logs matching Alerts raised and 620 records for Power logs

In [69]:
def unique_records(alert_matches_df, HighestValue):
    unique_alert_indices = alert_matches_df['alert_index'].unique()
    unique_alerts = pd.DataFrame()

    for index in unique_alert_indices:
        highest_value_record = HighestValue(index, alert_matches_df)
        if highest_value_record is not None:
            unique_alerts = unique_alerts.append(highest_value_record, ignore_index=True)

    return unique_alerts

In [70]:
TempAlerts = unique_records(Alert_Matches_Temp, HighestValue)
PowerAlerts = unique_records(Alert_Matches_Power, HighestValue)

/var/folders/2q/t2zqpdvs08xfcswxyd3rdsn80000gn/T/ipykernel_96739/857222388.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_alerts = unique_alerts.append(highest_value_record, ignore_index=True)
/var/folders/2q/t2zqpdvs08xfcswxyd3rdsn80000gn/T/ipykernel_96739/857222388.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_alerts = unique_alerts.append(highest_value_record, ignore_index=True)


In [71]:
TempAlerts.to_csv('TempAlerts.csv')
PowerAlerts.to_csv('PowerAlerts.csv')

In [72]:
TempAlerts

,timestamp,SITE,ASSET,COMPONENT,signal_name,VALUE,event_date,alert_index
0,1647112002197,-2073653851,-2126066089,-46972795,LM_handlePosCoreTempDegC,52.550562,2022-03-12,6
1,1647112095207,-2073653851,-2126066089,-46972795,LM_handlePosCoreTempDegC,59.486019,2022-03-12,7
2,1647112190202,-2073653851,-2126066089,-46972795,LM_handlePosCoreTempDegC,63.582844,2022-03-12,8
3,1647112312202,-2073653851,-2126066089,-46972795,LM_handlePosCoreTempDegC,69.322221,2022-03-12,9
4,1647178401444,-207607826,-1470107556,-837209140,LM_handlePosCoreTempDegC,49.258786,2022-03-13,178
...,...,...,...,...,...,...,...,...
615,1647724973480,-1844894889,1477164192,-1702932383,LM_handlePosCoreTempDegC,88.369624,2022-03-19,112
616,1647725087503,-1844894889,1477164192,-1702932383,LM_handlePosCoreTempDegC,91.233117,2022-03-19,113
617,1647662031276,-152918872,365951380,420204713,LM_handleNegCoreTempDegC,96.275534,2022-03-19,169
618,1647662166273,-152918872,365951380,420204713,LM_handleNegCoreTempDegC,101.675034,2022-03-19,170


In [73]:
MeanAlertTemp = TempAlerts['VALUE'].mean()

In [74]:
print(f"The mean value of Temperature when an Alert fires is : {MeanAlertTemp}")

The mean value of Temperature when an Alert fires is : 57.85496049002472


In [75]:
PowerAlerts

,timestamp,SITE,ASSET,COMPONENT,signal_name,VALUE,event_date,alert_index
0,1647112004196,-2073653851,-2126066089,-46972795,LM_powerLimit,298.749587,2022-03-12,6
1,1647112088207,-2073653851,-2126066089,-46972795,LM_powerLimit,288.530682,2022-03-12,7
2,1647112187186,-2073653851,-2126066089,-46972795,LM_powerLimit,267.344244,2022-03-12,8
3,1647112306182,-2073653851,-2126066089,-46972795,LM_powerLimit,242.840057,2022-03-12,9
4,1647178396429,-207607826,-1470107556,-837209140,LM_powerLimit,105.268708,2022-03-13,178
...,...,...,...,...,...,...,...,...
622,1647724971479,-1844894889,1477164192,-1702932383,LM_powerLimit,239.696659,2022-03-19,112
623,1647725088470,-1844894889,1477164192,-1702932383,LM_powerLimit,209.536231,2022-03-19,113
624,1647662027279,-152918872,365951380,420204713,LM_powerLimit,285.552574,2022-03-19,169
625,1647662165327,-152918872,365951380,420204713,LM_powerLimit,263.669798,2022-03-19,170


In [76]:
MeanAlertPower = PowerAlerts['VALUE'].mean()

In [77]:
print(f"The mean value of Power when an Alert fires is : {MeanAlertPower}")

The mean value of Power when an Alert fires is : 133.25195093840802


--------------------------------------------------------------------------------------------------------

In [78]:
Analyst_Alerts[Analyst_Alerts['COMPONENT']==1396988257]

,timestamp,SITE,ASSET,COMPONENT,alert_name,event_date
178,1647178401994,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-13
182,1647179349359,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-13
183,1647183761902,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-13
184,1647207629310,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-13
185,1647207729306,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-13
...,...,...,...,...,...,...
339,1647260636791,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-14
338,1647260632891,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-14
353,1647261103672,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-14
337,1647257144223,1524600080,-848050549,1396988257,LM_a141_cableTempImbalance,2022-03-14


In [79]:
concat_test = pd.DataFrame()
for key in dataframes_14:
    if "Temp" in key:
        concat_test = pd.concat([concat_test, dataframes_14[key]])

In [80]:
concat_test[concat_test['COMPONENT']==1396988257]

,timestamp,SITE,ASSET,COMPONENT,signal_name,VALUE,event_date
62700,1647247937125,1524600080,-848050549,1396988257,LM_pcbaTempDegC,36.743225,2022-03-14
62701,1647248233110,1524600080,-848050549,1396988257,LM_pcbaTempDegC,36.619531,2022-03-14
62702,1647248244120,1524600080,-848050549,1396988257,LM_pcbaTempDegC,37.395598,2022-03-14
62703,1647248255151,1524600080,-848050549,1396988257,LM_pcbaTempDegC,37.348207,2022-03-14
62704,1647248266109,1524600080,-848050549,1396988257,LM_pcbaTempDegC,37.311661,2022-03-14
...,...,...,...,...,...,...,...
65302,1647270289247,1524600080,-848050549,1396988257,LM_handleNegCoreTempDegC,-183.543050,2022-03-14
65503,1647272095164,1524600080,-848050549,1396988257,LM_handleNegCoreTempDegC,45.400621,2022-03-14
65515,1647272219154,1524600080,-848050549,1396988257,LM_handleNegCoreTempDegC,27.736654,2022-03-14
65518,1647272229163,1524600080,-848050549,1396988257,LM_handleNegCoreTempDegC,38.251060,2022-03-14


# Question 4

#### Workbook 1


1. From our visualisation we observe that over 50% of the records where the average temperature was recorded above 90 degrees for atleast 30 seconds came from 6 components - 399812473, 1348730178, -1473041981, 1996083301, 851777939, and 1036677714.


2. Roughly a quarter of all records flagged occurred on the 19th of March, 2022.


3. 4 Sites recorded over 2/3rds (68%) of the flagged records, significantly more than their proportional share of total records of 25%. Sites -1492083459, -152918872, -1576961961, 1226026637


4. The Maximum Powers recorded for Components corresponding where the Avg Temperature was above 90 degrees range from 204 to 360. This is in comparison to a median power of 91 recorded for all Components.

-----------------------------------------------------
#### Workbook 2


1. Over 75% of Alerts are recorded on the 2022-03-14. This is largely due to the disproportionate amount of Alerts cause by Component 1396988257.


2. Just two components account for over 90% of the Alerts fired over the week. Component 1396988257 and Component -837209140. 


3. Component 1396988257 accounts for almost 80% of the Alerts Fired over the entire week. Corresponding to the component, the Site 1524600080 and Asset -848050549 also account for similar perrcentages.


4. Component -837209140 accounts for almost 10% of the Alerts Fired over the entire week. Corresponding to the component, the Site -207607826 and Asset -1470107556 also account for similar perrcentages.


5. The Average Temperatue per Component when an Alert Fires ranges from 46 degrees to 100 degrees. Similarly the Average Power per Component ranges from 95 to 274.


6. However these figures are not entirely accurate or representative. The majority of Alerts were caused by Component 1396988257, however the number of records with the Component were proportionally much lower in the records identified as analogous to Alerts.


7. On further investigation we observed that the reason we were unable to identify analogous records for Component 1396988257 is that there were fewer Temperature logs on 2022-03-14 for the component compared to the Alerts. The temperatures for the component would fall down till -150 degrees and the records were not flagged.